In [476]:
import pandas as pd
import numpy as np
import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

# Создание новых признаков

Cчитаем таблицу с обработанными данными

In [451]:
data=pd.read_csv('df1.csv', sep=',')
data.head(5)

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,Latitude,Longitude,Lon_Lat,distance
0,261190187,2022-12-31,23:41:00,QUEENS,105,0,18-24,M,BLACK,<18,M,BLACK,40.693091,-73.733598,POINT (-73.733598 40.693091),0.029070
1,261175635,2022-12-30,16:26:00,BRONX,44,0,18-24,M,WHITE HISPANIC,45-64,M,BLACK,40.837369,-73.906691,POINT (-73.906691 40.837369),0.005830
2,261176929,2022-12-30,17:00:00,MANHATTAN,30,0,25-44,M,BLACK,25-44,M,BLACK,40.825160,-73.949519,POINT (-73.949519 40.82516),0.007743
3,261120108,2022-12-29,16:48:00,BRONX,42,1,18-24,F,BLACK,25-44,M,BLACK,40.830025,-73.908341,POINT (-73.908341 40.830025),0.002551
4,261120017,2022-12-29,22:08:00,QUEENS,101,0,18-24,M,BLACK,25-44,F,BLACK,40.603766,-73.759286,POINT (-73.759286 40.603766),0.066480


Создадим новый признак, который будет показывать, произошла стрельба ночью или нет. Кроме того, создадим признак, отвечающий за возраст жертвы (принадлежит к условной молодежи: 18-24 и 25-44, или нет)

In [452]:
data['year'] = pd.to_datetime(data['OCCUR_DATE']).dt.year
data['month'] = pd.to_datetime(data['OCCUR_DATE']).dt.month
data['day'] = pd.to_datetime(data['OCCUR_DATE']).dt.day
data['hour'] = pd.to_datetime(data['OCCUR_TIME']).dt.hour
data['is_night']=np.where((data['hour']==21)|(data['hour']==22)|(data['hour']==23)|(data['hour']==0)|(data['hour']==1)|(data['hour']==2), 1, 0)
data['is_young']=np.where((data['PERP_AGE_GROUP']=='18-24')|(data['PERP_AGE_GROUP']=='25-44'),1,0)
data.head(5)

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,...,Latitude,Longitude,Lon_Lat,distance,year,month,day,hour,is_night,is_young
0,261190187,2022-12-31,23:41:00,QUEENS,105,0,18-24,M,BLACK,<18,...,40.693091,-73.733598,POINT (-73.733598 40.693091),0.029070,2022,12,31,23,1,1
1,261175635,2022-12-30,16:26:00,BRONX,44,0,18-24,M,WHITE HISPANIC,45-64,...,40.837369,-73.906691,POINT (-73.906691 40.837369),0.005830,2022,12,30,16,0,1
2,261176929,2022-12-30,17:00:00,MANHATTAN,30,0,25-44,M,BLACK,25-44,...,40.825160,-73.949519,POINT (-73.949519 40.82516),0.007743,2022,12,30,17,0,1
3,261120108,2022-12-29,16:48:00,BRONX,42,1,18-24,F,BLACK,25-44,...,40.830025,-73.908341,POINT (-73.908341 40.830025),0.002551,2022,12,29,16,0,1
4,261120017,2022-12-29,22:08:00,QUEENS,101,0,18-24,M,BLACK,25-44,...,40.603766,-73.759286,POINT (-73.759286 40.603766),0.066480,2022,12,29,22,1,1


# Машинное обучение

Перед началом машинного обучения избавимся от ненужных колонок, таких как 'INCIDENT_KEY' (неинформатиный признак), 'OCCUR_DATE' (год и месяц выделили в отдельные признаки), 'OCCUR_TIME' (значение часа выделили в отдельный признак), 'PRECINCT' (неинформативен), 'Lon_Lat' (Longtitude и Latitude есть как отдельные признаки).

Мы хотим предсказывать умрет ли человек вследствие шутинга или нет, поэтому наш таргет - это колонка 'STATISTICAL_MURDER_FLAG' 

In [453]:
data=data.iloc[:,3:]
y=data['STATISTICAL_MURDER_FLAG']
X=data.drop(columns=['STATISTICAL_MURDER_FLAG','Lon_Lat','PRECINCT'])
X.head()

,BORO,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,Latitude,Longitude,distance,year,month,day,hour,is_night,is_young
0,QUEENS,18-24,M,BLACK,<18,M,BLACK,40.693091,-73.733598,0.029070,2022,12,31,23,1,1
1,BRONX,18-24,M,WHITE HISPANIC,45-64,M,BLACK,40.837369,-73.906691,0.005830,2022,12,30,16,0,1
2,MANHATTAN,25-44,M,BLACK,25-44,M,BLACK,40.825160,-73.949519,0.007743,2022,12,30,17,0,1
3,BRONX,18-24,F,BLACK,25-44,M,BLACK,40.830025,-73.908341,0.002551,2022,12,29,16,0,1
4,QUEENS,18-24,M,BLACK,25-44,F,BLACK,40.603766,-73.759286,0.066480,2022,12,29,22,1,1


In [444]:
X.shape

(17319, 17)

Мы решаем задачу классификации, поэтому будем использовать ROC-AUC для измерения качества модели. Для прогнозирования результатов воспользуемся CatBoost. Сначала разделим выборку на тест и трейн, затем с помощью grid_search подберем некоторые гиперпараметры: learning_rate, depth, l2_leaf_reg. Затем обучим модель с подобранными параметрами

In [454]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, train_size=0.8, random_state=42
)

In [455]:
cat_features = list(range(0, 7))+[11,12,13,14,15] #номера колонок с категориальными признаками

In [465]:
cat = CatBoostClassifier(n_estimators=200, cat_features=cat_features)

grid = {"learning_rate": [0.1, 0.12, 0.15], "depth": [4, 6, 8], 'l2_leaf_reg': [3, 5, 7, 9]}

grid_search_result = cat.grid_search(
    grid, X=X_train, y=y_train, verbose=False, plot=True, cv=5
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6474641	test: 0.6507418	best: 0.6507418 (0)	total: 31.8ms	remaining: 6.32s
1:	learn: 0.6104131	test: 0.6161468	best: 0.6161468 (1)	total: 64ms	remaining: 6.34s
2:	learn: 0.5787839	test: 0.5868211	best: 0.5868211 (2)	total: 101ms	remaining: 6.63s
3:	learn: 0.5598507	test: 0.5693345	best: 0.5693345 (3)	total: 115ms	remaining: 5.63s
4:	learn: 0.5415864	test: 0.5525711	best: 0.5525711 (4)	total: 151ms	remaining: 5.91s
5:	learn: 0.5272756	test: 0.5395032	best: 0.5395032 (5)	total: 194ms	remaining: 6.27s
6:	learn: 0.5180201	test: 0.5312255	best: 0.5312255 (6)	total: 209ms	remaining: 5.77s
7:	learn: 0.5102455	test: 0.5245887	best: 0.5245887 (7)	total: 278ms	remaining: 6.67s
8:	learn: 0.5015328	test: 0.5169554	best: 0.5169554 (8)	total: 330ms	remaining: 7s
9:	learn: 0.4962223	test: 0.5124953	best: 0.5124953 (9)	total: 370ms	remaining: 7.04s
10:	learn: 0.4904366	test: 0.5072033	best: 0.5072033 (10)	total: 410ms	remaining: 7.04s
11:	learn: 0.4876034	test: 0.5048222	best: 0.5048222 (1

In [468]:
grid_search_result["params"]

{'depth': 6, 'l2_leaf_reg': 9, 'learning_rate': 0.12}

Используем модель, которая автоматически остановливает процесс обучения, когда оценочная метрика на валидационном наборе данных перестает улучшаться

In [469]:
model_with_early_stop = CatBoostClassifier(
    eval_metric="AUC",
    iterations=200,
    random_seed=92,
    learning_rate=0.12,
    early_stopping_rounds=20,
    depth=6,
    l2_leaf_reg=9
)
model_with_early_stop.fit(
    X_train,
    y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=False,
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [478]:
pred=model_with_early_stop.predict_proba(X_validation)
p=model_with_early_stop.predict(X_validation)
print(f'ROC-AUC: {roc_auc_score(y_validation, pred[:,-1])}')
print(f'precision: {precision_score(y_validation, p)}') 
print(f'recall: {recall_score(y_validation, p)}') 
print(f'accuracy: {accuracy_score(y_validation,p)}')

ROC-AUC: 0.7530564619496012
precision: 0.49019607843137253
recall: 0.036337209302325583
accuracy: 0.8010969976905312


Посмотрим на признаки в зависимости от их важности для предсказания

In [471]:
model_with_early_stop.get_feature_importance(prettified=True)

,Feature Id,Importances
0,PERP_AGE_GROUP,31.696022
1,year,10.482502
2,PERP_SEX,7.253145
3,hour,6.876263
4,month,6.009470
5,BORO,5.437372
6,distance,5.250091
7,PERP_RACE,5.242636
8,VIC_AGE_GROUP,4.471480
9,day,4.446863


**Выводы:** ROC-AUC нашей модели составляет 0.75, т.е. модель имеет приемлемую способность различать классы. Однако, у нас очень низкий recall, то есть мы плохо находим положительный класс (связано с большим дисбалансом между классами). Кроме того, самыми важными призанками оказались раса шутера, год происшествия, пол шутера. 